In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

from sae.sparse_autoencoder import *
from sae.activation_store import *
from sae.train import ModelTrainer
from sae.config import create_config, log_config, Config
from sae.metrics import *
from sae.utils import get_blog_checkpoint, get_blog_sparsity

In [2]:
import sae_vis

In [3]:
from sae_vis.data_storing_fns import SaeVisData

In [4]:
from sae_vis.data_config_classes import SaeVisConfig
# from sae_vis.data_storing_fns import SaeVisData


In [137]:
from huggingface_hub import hf_hub_download
REPO_ID = "eoinf/pythia_saes"
# resid_pre_layer_24 :
FILENAME = "desert-oath-8/sparse_autoencoder_EleutherAI/pythia-2.8b-deduped_blocks.24.hook_resid_pre_s20480_92155904.pt"
# resid_pre_layer_10 :
# FILENAME = "absurd-dust-9/sparse_autoencoder_EleutherAI/pythia-2.8b-deduped_blocks.10.hook_resid_pre_s20480_92155904.pt"
# resid_pre_layer_11 (best one):
# FILENAME = "absurd-dust-9/sparse_autoencoder_EleutherAI//pythia-2.8b-deduped_blocks.11.hook_resid_pre_s20480_327675904.pt"

In [3]:
filename = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
sae = load_saved_sae(filename)


(…)cks.24.hook_resid_pre_s20480_92155904.pt:   0%|          | 0.00/420M [00:00<?, ?B/s]

In [4]:
model = model_store_from_sae(sae)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Loaded pretrained model EleutherAI/pythia-2.8b-deduped into HookedTransformer
Moving model to device:  cuda


In [5]:
activation_store = ActivationStoreAnalysis(sae.cfg, model)

buffer
dataloader


In [9]:
all_tokens = []
for _ in range(40):
    input_activations, input_tokens = activation_store.next_batch()
    all_tokens.append(input_tokens)


In [10]:
all_tokens = torch.cat(all_tokens).int().reshape(-1,128)

In [11]:
all_tokens = all_tokens.reshape(-1,128)

In [12]:
all_tokens.shape

torch.Size([1280, 128])

In [13]:
feature_vis_config_gpt = SaeVisConfig(
    hook_point=sae_l5.cfg.hook_point,
    features=np.arange(1000, 1100, 2),
    batch_size=128,
    minibatch_size_tokens=64,
    verbose=True,
)

In [14]:
sae_vis_data_gpt = SaeVisData.create(
    encoder=sae_l5,
    model=model,
    tokens=all_tokens,  # type: ignore
    cfg=feature_vis_config_gpt,
)

Forward passes to cache data for vis:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting vis data from cached data:   0%|          | 0/50 [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ Task                                           ┃ Time  ┃ Pct % ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ (1) Initialization                             │ 0.00s │ 0.0%  │
│ (2) Forward passes to gather model activations │ 0.08s │ 3.1%  │
│ (3) Computing feature acts from model acts     │ 0.63s │ 23.6% │
│ (4) Getting data for tables                    │ 0.01s │ 0.2%  │
│ (5) Getting data for histograms                │ 0.05s │ 1.7%  │
│ (6) Getting data for sequences                 │ 1.83s │ 68.6% │
│ (7) Getting data for quantiles                 │ 0.07s │ 2.7%  │
└────────────────────────────────────────────────┴───────┴───────┘

In [15]:
import os
import webbrowser

In [16]:
for feature in np.arange(1000, 1100, 2):
    filename = f"{feature}_feature_vis_demo_gpt.html"
    sae_vis_data_gpt.save_feature_centric_vis(filename, feature)
    webbrowser.open("file://" + os.path.abspath(filename))

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

...

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]

Saving feature-centric vis:   0%|          | 0/50 [00:00<?, ?it/s]